In [170]:
import pandas as pd
import numpy as np

def winProbability(eloA, eloB) :
    diff = eloA - eloB
    p = 1 - 1 / (1 + 10 ** (diff / 400.0))
    return(p)

def rate_1vs1(p1, p2, mov=1, week=8, drawn=False):
    k = .06 * (w - 17)**2 + 20
    
    k_multiplier = np.log(abs(mov) + 1)
    if mov < 0:
        k_multiplier *= -1
    corr_m = 2.2 / ((p1 - p2)*.001 + 2.2)
    rp1 = 10 ** (p1/400)
    rp2 = 10 ** (p2/400)
    exp_p1 = rp1 /(rp1 + rp2)
    exp_p2 = rp2 /(rp1 + rp2)
    if (drawn == True):
        s1 = 0.5
        s2 = 0.5
    else:
        s1 = 1
        s2 = 0
    new_p1 = p1 + k_multiplier * corr_m * k * (s1 - exp_p1)
    new_p2 = p2 + k_multiplier * corr_m * k * (s2 - exp_p2)
    return new_p1, new_p2
                                        
                                        


In [171]:
df = pd.read_csv("allSeasonScores.csv")
eloDict = dict()

# find average difference between Opoints and DPoints


avgPointsDict = dict()
for y in range(1999,2020):
    currSeason = df[df.season == y]
    HOPoints = np.array(currSeason.homeOffPoints)
    AOPoints = np.array(currSeason.awayOffPoints)
    HDPoints = np.array(currSeason.homeDefPoints)
    ADPoints = np.array(currSeason.awayDefPoints)
    homeMOV = np.mean(HOPoints - ADPoints)
    awayMOV = np.mean(AOPoints - HDPoints)
    avgPointsDict[y] = {'home' : homeMOV, 'away' : awayMOV}

In [172]:
'''
for team in set(df.homeTeam):
    eloDict[team] = dict()
    for y in range(1999,2020):
        eloDict[team][y] = dict()
        for w in range(0,18):
            eloDict[team][y][w] = dict()
            for unit in ['off', 'def']:
                eloDict[team][y][w][unit] = 10000000000000
'''
    
    #unit = {'off' : 1500, 'def' : 1500}
    #weekDict = {0: unit}
    #seasonDict = {1999: weekDict}
    #eloDict[team] = seasonDict

"\nfor team in set(df.homeTeam):\n    eloDict[team] = dict()\n    for y in range(1999,2020):\n        eloDict[team][y] = dict()\n        for w in range(0,18):\n            eloDict[team][y][w] = dict()\n            for unit in ['off', 'def']:\n                eloDict[team][y][w][unit] = 10000000000000\n"

In [173]:
def getMostRecentElo(team, unit):
    try: 
        season = eloDict[team][max(eloDict[team].keys())]
        result =  season[max(season.keys())][unit]
        #print(result)
        return result
    except:
        return 1500
    
    
def setDict(team, year, week):
    if eloDict[team] == None:
        eloDict[team] = dict()
    elif eloDict[team][year] == None:
        eloDict[team][year] = dict()
    elif eloDict[team][year][week] == None:
        eloDict[team][year][week] = dict()

In [174]:
maxO = 1500
minO = 1500
maxD = 1500
minD = 1500

eloDict = dict()
for index, row in df.iterrows():
    
    homeTeam = row.homeTeam
    awayTeam = row.awayTeam
    HomeOffDiff = row.homeOffPoints - row.awayDefPoints
    AwayOffDiff = row.awayOffPoints - row.homeDefPoints
    y = row.season
    w = row.week
    
    
    if homeTeam not in eloDict.keys():
        eloDict[homeTeam] = dict()
    if y not in eloDict[homeTeam].keys():
        eloDict[homeTeam][y] = {0 : {'off' : ((getMostRecentElo(homeTeam, 'off') - 1500) * 1/3) + 1500, 
                                     'def': ((getMostRecentElo(homeTeam, 'def') - 1500 * 1/3) + 1500)}}
        
    if awayTeam not in eloDict.keys():
        eloDict[awayTeam] = dict()
    if y not in eloDict[awayTeam].keys():
        eloDict[awayTeam][y] = {0 : {'off' : ((getMostRecentElo(awayTeam, 'off') - 1500) * 1/3) + 1500, 
                                     'def': ((getMostRecentElo(awayTeam, 'def') - 1500) * 1/3) + 1500}}
    
    avgHomeMOV = avgPointsDict[y]['home']
    avgAwayMOV = avgPointsDict[y]['away']
    HOELO = getMostRecentElo(homeTeam, 'off')
    ADELO = getMostRecentElo(awayTeam, 'def')
    a, b = rate_1vs1(HOELO, ADELO, HomeOffDiff - avgHomeMOV, w)
    
   

    eloDict[awayTeam][y][w] = dict()
    eloDict[homeTeam][y][w] = dict()
    
    eloDict[homeTeam][y][w]['off'] = a
    eloDict[awayTeam][y][w]['def'] = b
    
    avgMOV = avgPointsDict[y]
    
    HDELO = getMostRecentElo(awayTeam, 'off')
    AOELO = getMostRecentElo(homeTeam, 'def')
    c, d = rate_1vs1(AOELO, HDELO, AwayOffDiff - avgAwayMOV, w)
    eloDict[awayTeam][y][w]['off'] = c
    eloDict[homeTeam][y][w]['def'] = d
    
    
    maxO = max(maxO, a, c)
    minO = min(minO, a, c)
    maxD = max(maxD, b, d)
    minD = min(minD, b, d)

In [175]:
eloDict

{'PHI': {1999: {0: {'off': 1500.0, 'def': 2500.0},
   1: {'off': 1531.3167406228197, 'def': 1457.9468647204956},
   2: {'off': 1481.2505228415757, 'def': 1473.7624561668806},
   3: {'def': 1436.714039764897, 'off': 1454.8131234753148},
   4: {'def': 1467.7333168824152, 'off': 1463.5780224787973},
   5: {'off': 1427.375281084416, 'def': 1528.2897606761953},
   6: {'def': 1548.8870399275634, 'off': 1521.350311787786},
   7: {'def': 1583.893352514969, 'off': 1468.5809085749413},
   8: {'off': 1433.360012609797, 'def': 1492.76671519006},
   9: {'def': 1463.4087477648927, 'off': 1472.3048433756292},
   10: {'off': 1502.5251841568224, 'def': 1470.767300335923},
   11: {'off': 1475.847892766983, 'def': 1465.8577888991797},
   12: {'def': 1460.1796237695512, 'off': 1506.2556566135845},
   13: {'def': 1475.1340632715965, 'off': 1479.2963203989855},
   14: {'def': 1468.648880129536, 'off': 1479.7111842077845},
   15: {'off': 1500.0039238026388, 'def': 1521.306820324201},
   17: {'off': 1519.1054

In [180]:
# write the dictionary to a json file


import json

with open('eloVals.json', 'w') as fp:
    json.dump(eloDict, fp, sort_keys=True, indent=4)

In [182]:
with open('eloVals.json', 'r') as fp:
    testLoad = json.load(fp)

In [183]:
type(testLoad)

dict

In [ ]:

# team is the team abbreviation (ARI, ATL, BAL, etc)
# year is the season. Note that if we are talking about the 2018-19 season, year would be 2018
# week is which week we are looking for the ELO rating in.
#     *** NOTE *** this function assumes that you are attempting to find the ELO at the start of a week, meaning
#                  that if you passed 'week = 1' it would be the teams rating BEFORE the game was played. 
#                  this explains why it is '...[week - 1]...' in the 'try' part of the below function
# unit is either 'off' (offense) or 'def' (defense)
# data is the dictionary object
def getInitialEloRating(team, year, week, unit, data = eloDict):
    try:
        return data[team][year][week - 1][unit]
    except:
        # What if we are missing that week? For instance, the bucs dolphins game in 2017 week 1 was postponed
        # until week 11 because of a hurricane. So if you tried something like...
        # data['MIA'][2017][2]['off'] you would get an error...
        # the solution to this problem is that we are going to try and find the most recent ELO ratings.
        # it is guaranteed that for each team, for each season, there exists their week 0, or initial, elo ratings which
        # are based on their ending values from the prevrious season. So in the example above (bucs dolphins) we would end
        # up with the initial rank at week 0.
        season = data[team][year]
        
        # descending order...
        for w in range(week - 1,-1, -1):
            try:
                return season[w][unit]
            except:
                pass